# Prompt Engineering
In this notebook, we will explore the concept of prompt engineering.

## Structure of a Prompt
A prompt is a structured input that guides the model's response. It typically consists of:
- **Instruction**: A clear directive on what the model should do. Typically how it should use inputs and or external information to produce the desired output.
- **External information or context**: Additional data that we can manually provide the model prompt with, retrieve from a vector database, or retrieve from a web search.
- **User input**: the specific query that the user inputs.
- **Output indicator**: the beginning of the model's response, which can be a specific phrase or format that the model should follow. For example, "The answer is:" or "Response:"

Not all prompts will have all these components, but a well-structured prompt will typically include at least two or more of them.

In [1]:
from dotenv import load_dotenv
import warnings
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()

True

In [3]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Geoffrey Everest Hinton (born 6 December 1947) is a British-Canadian computer scientist, cognitive scientist, 
cognitive psychologist, known for his work on artificial neural networks which earned him the title as the 
"Godfather of AI". Hinton is University Professor Emeritus at the University of Toronto. From 2013 to 2023, 
he divided his time working for Google (Google Brain) and the University of Toronto, before publicly announcing 
his departure from Google in May 2023, citing concerns about the risks of artificial intelligence (AI) technology.

In 2017, he co-founded and became the chief scientific advisor of the Vector Institute in Toronto.
With David Rumelhart and Ronald J. Williams, Hinton was co-author of a highly cited paper published in 1986 
that popularised the backpropagation algorithm for training multi-layer neural networks, although they were 
not the first to propose the approach. Hinton is viewed as a leading figure in the deep learning community.
The image-recognition milestone of the AlexNet designed in collaboration with his students Alex Krizhevsky 
and Ilya Sutskever for the ImageNet challenge 2012[22] was a breakthrough in the field of computer vision.

Hinton received the 2018 Turing Award, often referred to as the "Nobel Prize of Computing", together with 
Yoshua Bengio and Yann LeCun, for their work on deep learning. They are sometimes referred to as the 
"Godfathers of Deep Learning", and have continued to give public talks together. He was also awarded 
the 2024 Nobel Prize in Physics, shared with John Hopfield.

Question: Which important achievements did he get?

Answer: """

In this example, we have:
* **Instruction**: "Answer the question based on the provided context."
* **External information or context**: The provided text about Hinton.
* **User input**: "Which contribution did he provide?"
* **Output indicator**: "Answer:"

In [4]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1,)

In [5]:
# make a response from our prompt
answer = llm.invoke(prompt)   
# print the response
print(answer.content.strip())

Geoffrey Hinton received the following important achievements:

1. The 2018 Turing Award, often referred to as the "Nobel Prize of Computing", together with Yoshua Bengio and Yann LeCun, for their work on deep learning.
2. The 2024 Nobel Prize in Physics, shared with John Hopfield.


We wouldn't typically know the user input in advance. So insted of writing the full prompt, we can use a template to create it dynamically.

In [6]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Geoffrey Everest Hinton (born 6 December 1947) is a British-Canadian computer scientist, cognitive scientist, 
cognitive psychologist, known for his work on artificial neural networks which earned him the title as the 
"Godfather of AI". Hinton is University Professor Emeritus at the University of Toronto. From 2013 to 2023, 
he divided his time working for Google (Google Brain) and the University of Toronto, before publicly announcing 
his departure from Google in May 2023, citing concerns about the risks of artificial intelligence (AI) technology.

In 2017, he co-founded and became the chief scientific advisor of the Vector Institute in Toronto.
With David Rumelhart and Ronald J. Williams, Hinton was co-author of a highly cited paper published in 1986 
that popularised the backpropagation algorithm for training multi-layer neural networks, although they were 
not the first to propose the approach. Hinton is viewed as a leading figure in the deep learning community.
The image-recognition milestone of the AlexNet designed in collaboration with his students Alex Krizhevsky 
and Ilya Sutskever for the ImageNet challenge 2012[22] was a breakthrough in the field of computer vision.

Hinton received the 2018 Turing Award, often referred to as the "Nobel Prize of Computing", together with 
Yoshua Bengio and Yann LeCun, for their work on deep learning. They are sometimes referred to as the 
"Godfathers of Deep Learning", and have continued to give public talks together. He was also awarded 
the 2024 Nobel Prize in Physics, shared with John Hopfield.

Question: {query}

Answer: """
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=prompt,
)

In [7]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


the output_parser is used to parse the model's response into a specific format.

In [8]:
chain = prompt_template | llm | output_parser

answer = chain.invoke({"query": "Which important achievements did he get?"})
print(answer)

Geoffrey Hinton received the following important achievements:

1. The 2018 Turing Award, often referred to as the "Nobel Prize of Computing", together with Yoshua Bengio and Yann LeCun, for their work on deep learning.
2. The 2024 Nobel Prize in Physics, shared with John Hopfield.


## Few-Shot Prompting. 
It is ideal for what we call _few-shot learning_ using our prompt. 
To give some context, the primary sources of knoledge for LLMs are:
+ __Parametric knowledge__: it is been learned during the training phase and is stored in the model's parameters.
+ __Source knowledge__: it is given to the input model at inference time, e.g via the prompt.

This means that we can use the prompt to provide the model with additional context or examples as _source knowledge_ that it can use to generate a more accurate response.



### Few-Shot Training

In [9]:
from IPython.display import Markdown
template = """Create a FQA in Markdown Format on the following questions and answers,
    Q1: Do you ship internationally?,
    A1: Yes, we ship worldwide. Shipping times and costs vary depending on the destination.
    Q2: How can I track my order?,
    A2: Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.
    Q3: What is your return policy?,
    A3: You can return items within 30 days of receipt for a full refund. Items must be in original condition and packaging.

    """
    

prompt_template = PromptTemplate(
    template=template,
    input_variables=[] # No input variables needed for this template
)

chain = prompt_template | llm | output_parser

answer = chain.invoke({})
Markdown(answer)

**Frequently Asked Questions (FAQ)**

### Shipping and Tracking

#### Q1: Do you ship internationally?
A1: **Yes, we ship worldwide.** Shipping times and costs vary depending on the destination.

#### Q2: How can I track my order?
A2: **Once your order ships, you’ll receive a tracking number via email.** You can also track orders through your account dashboard.

### Returns and Refunds

#### Q3: What is your return policy?
A3: **You can return items within 30 days of receipt for a full refund.** Items must be in original condition and packaging.

The output are ok but what if we want to format the output in aspecific way? We can give the model some examples of the desired output format in the prompt. This is called few-shot prompting.

In [11]:
template = """
Create a structured Markdown FAQ with links, headers, and bullet points based on the following questions and answers.

**Example:**

Example Input:
Q1: Do you ship internationally?
A1: Yes, we ship worldwide. Shipping times and costs vary depending on the destination.
Q2: How can I track my order?
A2: Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.
Q3: What is your return policy?
A3: You can return items within 30 days of receipt for a full refund. Items must be in original condition and packaging.

Example Output:
# Frequently Asked Questions
## Shipping
- **Do you ship internationally?**  
  Yes, we ship worldwide. Shipping times and costs vary depending on the destination.
- **How can I track my order?**  
  Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.
## Returns
- **What is your return policy?**  
  You can return items within 30 days of receipt for a full refund. Items must be in original condition and packaging.

Now, create a similar FAQ based on the following questions and answers:
Q1: What payment methods are accepted?,
A1: We accept Visa, MasterCard, PayPal, Apple Pay, and Klarna (Buy Now, Pay Later in select countries).,
Q2: Are your products eco-friendly?,
A2: Yes, we use sustainable materials and practices in our production process.,
Q3: Can I change or cancel my order after placing it?,
A3: We can modify or cancel orders within 1 hour of purchase. Contact our support team immediately for assistance.
"""

In [12]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=[] # No input variables needed for this template
)

chain = prompt_template | llm | output_parser

answer = chain.invoke({})
Markdown(answer)

# Frequently Asked Questions
## Payment and Ordering
- **What payment methods are accepted?**  
  We accept Visa, MasterCard, PayPal, Apple Pay, and Klarna (Buy Now, Pay Later in select countries).
- **Can I change or cancel my order after placing it?**  
  We can modify or cancel orders within 1 hour of purchase. Contact our support team immediately for assistance.

## Products and Sustainability
- **Are your products eco-friendly?**  
  Yes, we use sustainable materials and practices in our production process.

We got a more readable output by adding a few-shot example to the prompt.
In Langchain, we can also use FewShotPromptTemplate to create a few-shot prompt.

In [13]:
from langchain_core.prompts import FewShotPromptTemplate


In [14]:
examples = [
    {
        'input': (
            "Q1: Do you ship internationally?\n"
            "A1: Yes, we ship worldwide. Shipping times and costs vary depending on the destination.\n"
        ),
        'output': (
            "## Shipping\n"
            "- **Do you ship internationally?**  \n"
            "  Yes, we ship worldwide. Shipping times and costs vary depending on the destination.\n"
        )
    },
    {        'input': (
            "Q2: How can I track my order?\n"
            "A2: Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.\n"
        ),
        'output': (
            "## Tracking\n"
            "- **How can I track my order?**  \n"
            "  Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.\n"
        )
    },
    {
        'input': (
            "Q3: What is your return policy?\n"
            "A3: You can return items within 30 days of receipt for a full refund. Items must be in original condition and packaging.\n"
        ),
        'output': (
            "## Returns\n"
            "- **What is your return policy?**  \n"
            "  You can return items within 30 days of receipt for a full refund. Items must be in original condition and packaging.\n"
        )
    }
]

In [15]:
# Create template 
example_template = """Example input:
{input}

Example Output:
# Frequently Asked Questions\n\n
{output}
"""

In [16]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template
)

In [17]:
# Instruction prefix
prefix = """Create a structured Markdown FAQ with links, headers, and bullet points based on the following questions and answers.

**Example:**"""

# User input format and instructions
suffix = """
Now, create a similar FAQ based on the following questions and answers:
{input}"""

In [18]:
# Create the few-shot prompt template
few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],
    example_separator="\n"
)

In [19]:
user_input = """Q1: What payment methods are accepted?,
A1: We accept Visa, MasterCard, PayPal, Apple Pay, and Klarna (Buy Now, Pay Later in select countries).,
Q2: Are your products eco-friendly?,
A2: Yes, we use sustainable materials and practices in our production process.,
Q3: Can I change or cancel my order after placing it?,
A3: We can modify or cancel orders within 1 hour of purchase. Contact our support team immediately for assistance.
"""

Now let's see what prompt this creates when we feed in a user query...

In [20]:
formatted_prompt = few_shot.format(input=user_input)
print(formatted_prompt)

Create a structured Markdown FAQ with links, headers, and bullet points based on the following questions and answers.

**Example:**
Example input:
Q1: Do you ship internationally?
A1: Yes, we ship worldwide. Shipping times and costs vary depending on the destination.


Example Output:
# Frequently Asked Questions


## Shipping
- **Do you ship internationally?**  
  Yes, we ship worldwide. Shipping times and costs vary depending on the destination.


Example input:
Q2: How can I track my order?
A2: Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.


Example Output:
# Frequently Asked Questions


## Tracking
- **How can I track my order?**  
  Once your order ships, you’ll receive a tracking number via email. You can also track orders through your account dashboard.


Example input:
Q3: What is your return policy?
A3: You can return items within 30 days of receipt for a full refund. Items must be in original condi

We can use a chain to generate responses

In [21]:
chain = few_shot | llm | output_parser

answer = chain.invoke({"input": user_input})
Markdown(answer)

# Frequently Asked Questions

## Payment and Ordering
- **What payment methods are accepted?**  
  We accept Visa, MasterCard, PayPal, Apple Pay, and Klarna (Buy Now, Pay Later in select countries).

## Sustainability
- **Are your products eco-friendly?**  
  Yes, we use sustainable materials and practices in our production process.

## Order Management
- **Can I change or cancel my order after placing it?**  
  We can modify or cancel orders within 1 hour of purchase. Contact our support team immediately for assistance.

Note: This FAQ is a basic structure and can be expanded to include more questions and answers as needed.

## Conclusion
In this notebook, we have learnt:
- how we can use prompt engineering to guide the model's response
- how to structure a prompt with an instruction, external information, user input, and output indicator
- how to make use of few-shot prompting to provide examples of the desired output format
- how to use LangChain's FewShotPromptTemplate to create a few-shot prompt